In [1]:
#import libraries
import requests
import pandas as pd
import time
import numpy as np 

In [8]:
#API call for top players
def get_top_player_ids():
    API_KEY = "RGAPI-4912c48d-414b-4c5f-9412-df8f28839e03"
    url = "https://na1.api.riotgames.com/tft/league/v1/challenger?api_key="+API_KEY
    response_top_players = requests.get(url).json()

    summonerIds = []
    for entry in response_top_players['entries']:
        summonerIds.append(entry['summonerId'])
    print("Completed retriving summonerIds")
    return summonerIds

In [9]:
#API call for the top player's list of match ids
def get_matches_ids():
    API_KEY = "RGAPI-4912c48d-414b-4c5f-9412-df8f28839e03"
    max_calls = 2
    puuids = []
    summonerIds = get_top_player_ids()
    start_time = time.time()
    time.sleep(120)
    
    for id in summonerIds:
        if max_calls % 100 == 0:
            time.sleep(120)
        elif max_calls % 20 == 0:
            time.sleep(2)
        url = "https://na1.api.riotgames.com/tft/summoner/v1/summoners/" + id + "?api_key="+API_KEY
        response_top_players_id = requests.get(url).json()
        #if len(response_top_players_id.keys()) > 2:
        puuids.append(response_top_players_id['puuid'])
        max_calls += 1

    max_calls = 1
    match_ids = []
    time.sleep(120)
    
    for id in puuids:
        if max_calls % 100 == 0:
            time.sleep(120)
        elif max_calls % 20 == 0:
            time.sleep(2)
        url = "https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/" + id + "/ids?start=0&count=20&api_key="+API_KEY
        response_top_players_match_id = requests.get(url).json()
        #if isinstance(response_top_players_match_id, list):
        match_ids.extend(response_top_players_match_id)
    match_ids = list(dict.fromkeys(match_ids))
    
    end_time = time.time()
    time_lapsed = end_time - start_time
    print("Completed retriving matchIds " + str(time_lapsed) + " seconds")
    return match_ids

In [10]:
#API call for matches
def get_matches():
    API_KEY = "RGAPI-4912c48d-414b-4c5f-9412-df8f28839e03"
    max_match_calls = 1
    match_infomation = []
    match_ids = get_matches_ids()
    start_time = time.time()
    time.sleep(120)
    
    for id in match_ids:
        if max_match_calls % 100 == 0:
            time.sleep(120)
        elif max_match_calls % 20 == 0:
            time.sleep(2)
        url = "https://americas.api.riotgames.com/tft/match/v1/matches/" + id + "/?api_key="+API_KEY
        response_matches = requests.get(url).json()
        #if len(response_matches.keys()) > 2:
        match_infomation.append(response_matches)
        max_match_calls += 1
    
    end_time = time.time()
    time_lapsed = end_time - start_time
    print("Completed retriving match_infomation after " + str(time_lapsed) + " seconds")
    return match_infomation

In [5]:
#Function that returns the game outcomes 
def get_units(df, match_infomation):
    game = 0
    for match in match_infomation:
        participants = match['info']['participants']
        for participant in participants:
            units = participant['units']
            total_units = 0
            for unit in units:
                if "Nomsy" in unit['character_id']:
                    total_units+=1
                elif ("Dragon" in unit['character_id']) | ("AoShin" in unit['character_id']) | ("AurelionSol" in unit['character_id']) | ("Shyvana" in unit['character_id']): 
                    total_units+=2
                else:
                    total_units+=1
            for unit in units:
                unit_name = unit['character_id']
                num_items = len(unit['items'])
                tier = unit['tier']
                placement = participant['placement']

                df = df.append({'unit_name': unit_name,
                                'num_items': num_items,
                                'tier': tier,
                                'placement': placement,
                               'game': game,
                               'total_units': total_units}, ignore_index=True)
        game += 1
    return df

In [11]:
#Main
df = pd.DataFrame(columns=["unit_name", "num_items", "tier", "placement", "game", "total_units"])
match_infomation = get_matches()
df = get_units(df, match_infomation)
df.to_csv("game_outcomes.csv")

Completed retriving summonerIds
Completed retriving matchIds 291.8899929523468 seconds
Completed retriving match_infomation after 1073.442409992218 seconds
